In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%%capture
!pip install transformers[torch]
!pip install kaggle
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [3]:
! kaggle competitions download -c detecting-french-texts-difficulty-level-2023
from zipfile import ZipFile
with ZipFile('detecting-french-texts-difficulty-level-2023.zip','r') as zip:
  zip.extractall(path="")

  0% 0.00/303k [00:00<?, ?B/s]
100% 303k/303k [00:00<00:00, 104MB/s]


In [4]:
import pandas as pd
import numpy as np
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [5]:
import torch

In [21]:
unlabelled_data = pd.read_csv('unlabelled_test_data.csv', index_col = 'id')

In [7]:
new_model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Data Science and Machine Learning/Project_Data_Science/train_model_bert/')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Data Science and Machine Learning/Project_Data_Science/train_model_bert/ and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
%%capture
new_tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-french-europeana-cased")

In [9]:
def get_multiclass_predictions(texts):
    encodings = new_tokenizer(texts.tolist(), return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    encodings = {k: v.to(new_model.device) for k, v in encodings.items()}

    with torch.no_grad():
        outputs = new_model(**encodings)

    logits = outputs.logits

    softmax = torch.nn.Softmax(dim=-1)  # Adjust the dimension to -1
    probabilities = softmax(logits.squeeze().cpu())
    probabilities = probabilities.detach().numpy()

    labels = np.argmax(probabilities, axis=-1)

    label_mapping = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}

    predicted_labels = [label_mapping[label.item()] for label in labels]

    return predicted_labels

In [10]:
#test = unlabelled_data.head(2)

In [11]:
#test

In [12]:
#get_multiclass_predictions(test['sentence'])

In [13]:
predicted_labels = get_multiclass_predictions(unlabelled_data['sentence'])
#predictions = pd.DataFrame({'difficulty': predicted_labels})

In [22]:
unlabelled_data['difficulty'] = predicted_labels

In [26]:
predictions = unlabelled_data[['difficulty']]

In [27]:
predictions.head()

,difficulty
id,
0,A2
1,A2
2,A2
3,A2
4,A2


In [28]:
predictions.to_csv('submission.csv')

In [29]:
! kaggle competitions submit -c detecting-french-texts-difficulty-level-2023 -f submission.csv -m "Sample submission"

100% 8.30k/8.30k [00:00<00:00, 10.1kB/s]
Successfully submitted to Detecting the difficulty level of French texts